# Генерация датасетов для обучения LLM

Этот notebook показывает как генерировать датасеты для SFT (Supervised Fine-Tuning) обучения.

**Возможности:**
- **81 тип задач** (40 математических + 41 физическая)
- Задачи на reasoning: Судоку, Загадки Эйнштейна, Переправы, Ханойская башня, Кувшины, Игра Ним
- **Новые физические задачи:** Баллистика, Вращение, Центр масс, Машина Атвуда, Наклонная плоскость, Индукция, RLC-цепи, RC-цепи, Сила Лоренца, Циклы Карно, Энтропия, Фазовые переходы, Доплер, Интерференция, Дифракция, Поляризация, Модель Бора, Де Бройль, Принцип неопределённости, Радиоактивный распад, Анализ размерностей, Погрешности, Перевод единиц
- Двуязычность (ru/en)
- 10 уровней сложности
- Форматы: SFT (Alpaca), Chat, JSON, JSONL

In [1]:
# Импорты
from re_rl.dataset_generator import DatasetGenerator
from re_rl.tasks import (
    ArithmeticTask, QuadraticTask, LinearTask,
    KinematicsTask, QuantumTask, NuclearTask,
    ALL_TASK_GENERATORS
)
from re_rl.tasks.physics.generators import ALL_PHYSICS_TASK_GENERATORS

# Задачи на reasoning
from re_rl.tasks.math.logic import SudokuTask, ZebraPuzzleTask
from re_rl.tasks.math.planning import RiverCrossingTask, TowerOfHanoiTask, WaterJugTask
from re_rl.tasks.math.discrete import NimGameTask

# Новые физические задачи
from re_rl.tasks.physics.mechanics import (
    ProjectileMotionTask, RotationalDynamicsTask, CenterOfMassTask,
    AtwoodMachineTask, InclinedPlaneTask
)
from re_rl.tasks.physics.electricity import (
    ElectromagneticInductionTask, ACCircuitsTask, RCCircuitsTask
)
from re_rl.tasks.physics.magnetism import MagneticForceTask
from re_rl.tasks.physics.thermodynamics import (
    ThermodynamicCyclesTask, EntropyTask, PhaseTransitionsTask
)
from re_rl.tasks.physics.waves import (
    DopplerEffectTask, InterferenceTask, DiffractionTask, PolarizationTask
)
from re_rl.tasks.physics.quantum import (
    BohrModelTask, DeBroglieTask, UncertaintyPrincipleTask, RadioactiveDecayTask
)
from re_rl.tasks.physics.measurements import (
    DimensionalAnalysisTask, ErrorPropagationTask, UnitConversionTask
)

## 1. Быстрый старт: генерация SFT датасета

In [2]:
generator = DatasetGenerator()

# Генерация SFT датасета
dataset = generator.generate_sft_dataset(
    task_types=["quadratic", "kinematics", "quantum"],
    num_samples=1000,
    language="ru",
    difficulties=[3, 5, 7],
)

print(f"Сгенерировано: {len(dataset)} примеров")
print(f"\nПример:")
print(f"instruction: {dataset[0]['instruction']}")
print(f"input: {dataset[0]['input'][:100]}...")
print(f"output: {dataset[0]['output'][:200]}...")

Генерация задач: 100%|██████████| 999/999 [00:01<00:00, 741.91задач/s]

Сгенерировано: 999 примеров

Пример:
instruction: Решите задачу пошагово, объясняя каждый шаг рассуждения.
input: Решите квадратное уравнение: x² - 8x + 15 = 0...
output: Шаг 1: Записываем уравнение в стандартной форме:
x² - 8x + 15 = 0
Шаг 2: Анализируем уравнение:
- Коэффициент при x²: 1
- Коэффициент при x: -8
- Свободный член: 15
Шаг 3: Находим корни уравнения: 3, ...


## 2. Все доступные типы задач

In [3]:
# Все 81 типов задач теперь в ALL_TASK_GENERATORS
all_tasks = sorted(ALL_TASK_GENERATORS.keys())

# Разделим по категориям
math_tasks = [t for t in all_tasks if t not in ALL_PHYSICS_TASK_GENERATORS]
physics_tasks = list(ALL_PHYSICS_TASK_GENERATORS.keys())

print("МАТЕМАТИЧЕСКИЕ ЗАДАЧИ:")
for i, name in enumerate(sorted(math_tasks), 1):
    print(f"  {i:2}. {name}")
print(f"\nВсего математических: {len(math_tasks)}")

print("\nФИЗИЧЕСКИЕ ЗАДАЧИ:")
for i, name in enumerate(sorted(physics_tasks), 1):
    print(f"  {i:2}. {name}")
print(f"\nВсего физических: {len(physics_tasks)}")

print(f"\n=== ИТОГО: {len(ALL_TASK_GENERATORS)} типов задач ===")

МАТЕМАТИЧЕСКИЕ ЗАДАЧИ:
   1. analogical
   2. arithmetic
   3. calculus
   4. category_theory
   5. combinatorics
   6. complex_number
   7. contradiction
   8. cubic
   9. differential_equation
  10. exponential
  11. financial_math
  12. futoshiki
  13. geometry
  14. graph
  15. group_theory
  16. inequality
  17. integral
  18. knights_knaves
  19. limits
  20. linear
  21. logarithmic
  22. matrix
  23. nim_game
  24. number_theory
  25. optimization
  26. quadratic
  27. river_crossing
  28. sequence
  29. series
  30. set_logic
  31. statistics
  32. sudoku
  33. system_linear
  34. text_stats
  35. tower_of_hanoi
  36. trigonometry
  37. urn_probability
  38. vector_3d
  39. water_jug
  40. zebra_puzzle

Всего математических: 40

ФИЗИЧЕСКИЕ ЗАДАЧИ:
   1. ac_circuits
   2. astrophysics
   3. atwood_machine
   4. bohr_model
   5. capacitors
   6. center_of_mass
   7. circuits
   8. de_broglie
   9. diffraction
  10. dimensional_analysis
  11. doppler_effect
  12. dynamics
  13. e

## 3. Примеры задач с разной сложностью

In [4]:
print("=" * 60)
print("ArithmeticTask — арифметические задачи")
print("=" * 60)

for difficulty in [1, 5, 10]:
    task = ArithmeticTask(difficulty=difficulty, language="ru")
    result = task.get_result()
    print(f"\nСложность {difficulty}:")
    print(f"  Задача: {result['problem']}")
    print(f"  Ответ: {result['final_answer']}")

ArithmeticTask — арифметические задачи

Сложность 1:
  Задача: Вычислите: 9 + 10
  Ответ: 19

Сложность 5:
  Задача: Вычислите: 30811485375 ÷ (33 × 95)^3
  Ответ: 1

Сложность 10:
  Задача: Вычислите: ((116 + 889) ÷ (685 + 8/7 - 222) + 221 × 497 + 224 × 938 × 64 - (146 + (25% - (765 - (28 - 24))) × (766 + 75% - 730))) × (2/7 - 910 ÷ 941)
  Ответ: -9.25591e+06


In [5]:
print("=" * 60)
print("Квантовая механика — фотоэффект")
print("=" * 60)

task = QuantumTask(task_type="photoelectric", difficulty=5, language="ru", detail_level=5)
task.solve()

print(f"\nЗадача: {task.description}")
print(f"\nШаги решения:")
for i, step in enumerate(task.solution_steps, 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.final_answer}")

Квантовая механика — фотоэффект

Задача: Фотон с частотой 8.185e+15 Гц падает на металл с работой выхода 3.431575398996858 эВ. Найдите кинетическую энергию фотоэлектрона.

Шаги решения:
  1. Шаг 1: Уравнение Эйнштейна для фотоэффекта: hν = A + Eₖ
  2. E_фотона = hν = 6.626e-34 × 8.185e+15 = 5.423e-18 Дж = 33.85 эВ
  3. Eₖ = hν - A = 33.85 - 3.431575398996858 = 30.42 эВ

Ответ: 30.4188 эВ


In [6]:
print("=" * 60)
print("Ядерная физика — энергия связи")
print("=" * 60)

task = NuclearTask(task_type="binding_energy", nucleus="He-4", language="ru", detail_level=10)
task.solve()

print(f"\nЗадача: {task.description}")
print(f"\nШаги решения:")
for i, step in enumerate(task.solution_steps, 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.final_answer}")

Ядерная физика — энергия связи

Задача: Найдите энергию связи ядра He-4 (A = 4, Z = 2).

Шаги решения:
  1. Шаг 1: Eсв = Δm·c² = [Z·m_p + (A-Z)·m_n - M]·931.5 МэВ
  2. m_теор = 2×1.007276 + 2×1.008665 = 4.031882 а.е.м.
  3. Δm = 4.031882 - 4.002603 = 0.029279 а.е.м.
  4. Eсв = 0.029279 × 931.5 = 27.27 МэВ
  5. Eсв/A = 27.27/4 = 6.82 МэВ/нуклон

Ответ: Eсв = 27.27 МэВ (6.82 МэВ/нуклон)


## 4. Новые задачи на Reasoning

Задачи, специально разработанные для обучения LLM рассуждениям и RL.

In [7]:
print("=" * 60)
print("SudokuTask — Судоку (4x4 и 9x9)")
print("=" * 60)

task = SudokuTask(difficulty=3, language="ru", detail_level=3)
result = task.get_result()

print(f"\nЗадача:\n{result['problem'][:500]}...")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:3], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет:\n{result['final_answer']}")

SudokuTask — Судоку (4x4 и 9x9)

Задача:
type: structured_text_with_tags
Описание: Решите головоломку судоку.
Правила:
- Каждая строка должна содержать числа от 1 до N без повторений
- Каждый столбец должен содержать числа от 1 до N без повторений
- Каждый блок должен содержать числа от 1 до N без повторений
Формат ответа:
  <reasoning>
    (Шаги решения)
  </reasoning>
  <answer>
    (Заполненная сетка)
  </answer>

Решите судоку размера 4x4:

. . | 3 1
3 . | . .
---------
2 . | 1 .
. 3 | 4 2

Пустые клетки обозначены точкой (.)....

Шаги решения:
  1. Naked single: клетка (1, 1) может содержать только 4.
  2. Naked single: клетка (1, 2) может содержать только 2.
  3. Naked single: клетка (2, 2) может содержать только 1.

Ответ:
Решение судоку:
4 2 | 3 1
3 1 | 2 4
---------
2 4 | 1 3
1 3 | 4 2


In [8]:
print("=" * 60)
print("ZebraPuzzleTask — Загадка Эйнштейна")
print("=" * 60)

task = ZebraPuzzleTask(difficulty=3, language="ru", detail_level=3)
result = task.get_result()

print(f"\nЗадача:\n{result['problem'][:600]}...")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:3], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет: {task.answer}")

ZebraPuzzleTask — Загадка Эйнштейна

Задача:
type: structured_text_with_tags
Описание: Решите логическую головоломку типа 'Загадка Эйнштейна'.
Используйте метод исключения и дедукцию для определения всех соответствий.
Формат ответа:
  <reasoning>
    (Пошаговые логические выводы)
  </reasoning>
  <answer>
    (Таблица соответствий)
  </answer>

Есть 4 домов в ряд, пронумерованных слева направо от 1 до 4.
В каждом доме живёт человек с уникальными характеристиками.

Категории: nationality, color, drink

Подсказки:
1. Человек с зелёный живёт рядом с человеком с молоко.
2. Человек с красный живёт слева от человека с жёлтый.
3. Человек с белы...

Шаги решения:
  1. Из подсказки 1: зелёный и молоко — соседи
  2. Из подсказки 2: красный левее жёлтый
  3. Из подсказки 3: белый в доме 1

Ответ: швед


In [9]:
print("=" * 60)
print("RiverCrossingTask — Задача о переправе")
print("=" * 60)

task = RiverCrossingTask(difficulty=1, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:5], 1):
    print(f"  {i}. {step}")
print(f"\nОтвет:\n{result['final_answer']}")

RiverCrossingTask — Задача о переправе

Задача:
type: structured_text_with_tags
Описание: Решите задачу о переправе через реку.
Найдите последовательность действий, чтобы переправить всех на другой берег.
Формат ответа:
  <reasoning>
    (Анализ ограничений и план)
  </reasoning>
  <answer>
    (Последовательность переправ)
  </answer>

Крестьянин должен переправить через реку волка, козу и капусту.
Лодка вмещает только крестьянина и один предмет.
Ограничения:
- Волк съест козу, если оставить их одних
- Коза съест капусту, если оставить их одних

Найдите последовательность переправ.

Шаги решения:
  1. Начальное состояние: левый берег = крестьянин, волк, коза, капуста, правый берег = -, лодка на левый берегу.
  2. Переправа 1: коза переправляются на правый берег.
  3. После переправы: левый = волк, капуста, правый = крестьянин, коза.
  4. Переправа 2: крестьянин переправляются на левый берег.
  5. После переправы: левый = крестьянин, волк, капуста, правый = коза.

Ответ:
Решение за 7 п

In [10]:
print("=" * 60)
print("TowerOfHanoiTask — Ханойская башня")
print("=" * 60)

task = TowerOfHanoiTask(difficulty=2, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nКоличество дисков: {task.num_disks}")
print(f"Оптимальное количество ходов: 2^{task.num_disks} - 1 = {2**task.num_disks - 1}")
print(f"\nПервые 7 ходов:")
for i, (disk, from_peg, to_peg) in enumerate(task.moves[:7], 1):
    print(f"  {i}. Диск {disk}: {from_peg} → {to_peg}")

TowerOfHanoiTask — Ханойская башня

Задача:
type: structured_text_with_tags
Описание: Решите задачу 'Ханойская башня'.
Переместите все диски с начального стержня на конечный.
Правила:
- Можно перемещать только один диск за раз
- Нельзя класть больший диск на меньший
Формат ответа:
  <reasoning>
    (Стратегия решения)
  </reasoning>
  <answer>
    (Последовательность ходов)
  </answer>

Дано 3 дисков разного размера на стержне A.
Диски расположены по убыванию размера снизу вверх (самый большой внизу).
Переместите все диски на стержень C, используя стержень B как вспомогательный.

Найдите оптимальную последовательность ходов.

Количество дисков: 3
Оптимальное количество ходов: 2^3 - 1 = 7

Первые 7 ходов:
  1. Диск 1: A → C
  2. Диск 2: A → B
  3. Диск 1: C → B
  4. Диск 3: A → C
  5. Диск 1: B → A
  6. Диск 2: B → C
  7. Диск 1: A → C


In [11]:
print("=" * 60)
print("WaterJugTask — Задача о кувшинах")
print("=" * 60)

task = WaterJugTask(difficulty=1, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nОбъёмы кувшинов: {task.capacities}")
print(f"Цель: {task.target} литров")
print(f"Разрешима: {task.is_solvable}")
print(f"\nШаги решения:")
for i, step in enumerate(result['solution_steps'][:5], 1):
    print(f"  {i}. {step}")

WaterJugTask — Задача о кувшинах

Задача:
type: structured_text_with_tags
Описание: Решите задачу о кувшинах с водой.
Используя кувшины заданных объёмов, отмерьте ровно нужное количество воды.
Доступные операции: наполнить, опустошить, перелить.
Формат ответа:
  <reasoning>
    (Стратегия и анализ)
  </reasoning>
  <answer>
    (Последовательность действий)
  </answer>

Даны кувшины объёмом 3, 5 литров.
Начальное состояние: все кувшины пусты.
Источник воды неограничен.

Цель: отмерить ровно 4 литров воды.

Найдите последовательность действий.

Объёмы кувшинов: (3, 5)
Цель: 4 литров
Разрешима: True

Шаги решения:
  1. Подсказка: НОД(3, 5) = 1. Цель 4 достижима, т.к. 4 делится на 1.
  2. Шаг 1: Наполнить кувшин 2 (5 л). Состояние: J1=0л, J2=5л.
  3. Шаг 2: Перелить из кувшина 2 в кувшин 1. Состояние: J1=3л, J2=2л.
  4. Шаг 3: Опустошить кувшин 1. Состояние: J1=0л, J2=2л.
  5. Шаг 4: Перелить из кувшина 2 в кувшин 1. Состояние: J1=2л, J2=0л.


In [12]:
print("=" * 60)
print("NimGameTask — Игра Ним (теория игр)")
print("=" * 60)

task = NimGameTask(difficulty=4, language="ru", detail_level=5)
result = task.get_result()

print(f"\nЗадача:\n{result['problem']}")
print(f"\nКучки: {task.piles}")
print(f"Ним-сумма (XOR): {task.nim_sum}")
if task.winning_move:
    pile_idx, take, new_size = task.winning_move
    print(f"Выигрышный ход: взять {take} из кучки {pile_idx + 1}")
else:
    print("Позиция проигрышная")
print(f"\nОтвет:\n{result['final_answer']}")

NimGameTask — Игра Ним (теория игр)

Задача:
type: structured_text_with_tags
Описание: Проанализируйте позицию в игре Ним.
В игре Ним два игрока по очереди берут предметы из кучек.
За один ход можно взять любое количество предметов из одной кучки.
Проигрывает тот, кто берёт последний предмет (или не может сделать ход).
Формат ответа:
  <reasoning>
    (Анализ с использованием XOR/Sprague-Grundy)
  </reasoning>
  <answer>
    (Выигрышный ход или вывод о позиции)
  </answer>

Игра Ним: есть кучки с 5, 3, 7 предметами.
Сейчас ход первого игрока.

Кто победит при оптимальной игре? Если первый игрок побеждает, укажите выигрышный ход.

Кучки: [5, 3, 7]
Ним-сумма (XOR): 1
Выигрышный ход: взять 1 из кучки 1

Ответ:
Первый игрок побеждает.
Выигрышный ход: взять 1 предмет(ов) из кучки 1.
После хода: 4, 3, 7.


## 5. Новые физические задачи (23 задачи)

Механика, электромагнетизм, термодинамика, волны/оптика, современная физика, измерения.

In [13]:
print("=" * 60)
print("МЕХАНИКА")
print("=" * 60)

# Баллистика
print("\n--- ProjectileMotionTask (Баллистика) ---")
task = ProjectileMotionTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Вращательное движение
print("\n--- RotationalDynamicsTask (Вращение) ---")
task = RotationalDynamicsTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Центр масс
print("\n--- CenterOfMassTask (Центр масс) ---")
task = CenterOfMassTask(difficulty=3, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Машина Атвуда
print("\n--- AtwoodMachineTask (Машина Атвуда) ---")
task = AtwoodMachineTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Наклонная плоскость
print("\n--- InclinedPlaneTask (Наклонная плоскость) ---")
task = InclinedPlaneTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

МЕХАНИКА

--- ProjectileMotionTask (Баллистика) ---
Задача: Решите задачу на движение тела, брошенного под углом к горизонту.

Тело брошено под углом 55.6° к горизонту с начальной скоростью 47.2 м/с. Найдите время полёта.
Ответ: Ответ: T = 7.94 с

--- RotationalDynamicsTask (Вращение) ---
Задача: Решите задачу на вращательное движение.

Найдите момент импульса тела с моментом инерции I = 1.42 кг·м² и угловой скоростью ω = 19.77 рад/с.
Ответ: Ответ: L = 28.07 кг·м²/с

--- CenterOfMassTask (Центр масс) ---
Задача: Найдите координаты центра масс системы тел.

Даны 3 тел с массами 3.8, 1.3, 9.7 кг в точках с координатами (-0.4, 8.7), (4.1, 10.0), (1.0, -7.2). Найдите центр масс системы.
Ответ: Центр масс: (0.913, -1.607) м

--- AtwoodMachineTask (Машина Атвуда) ---
Задача: Решите задачу на систему блоков (машина Атвуда).

Два груза массами m₁ = 10.6 кг и m₂ = 5.6 кг связаны нерастяжимой нитью, перекинутой через невесомый блок. Найдите ускорение системы и натяжение нити.
Ответ: Ответ: a = 3

In [14]:
print("=" * 60)
print("ЭЛЕКТРОМАГНЕТИЗМ")
print("=" * 60)

# Электромагнитная индукция
print("\n--- ElectromagneticInductionTask (Индукция) ---")
task = ElectromagneticInductionTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Цепи переменного тока
print("\n--- ACCircuitsTask (RLC-цепи) ---")
task = ACCircuitsTask(task_type="impedance", difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# RC-цепи
print("\n--- RCCircuitsTask (RC-цепи) ---")
task = RCCircuitsTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Сила Лоренца
print("\n--- MagneticForceTask (Сила Лоренца) ---")
task = MagneticForceTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

ЭЛЕКТРОМАГНЕТИЗМ

--- ElectromagneticInductionTask (Индукция) ---
Задача: Решите задачу на электромагнитную индукцию.

Магнитный поток через контур изменяется по закону Φ(t) = 0.281·0.112·sin(454.9·t) Вб. Найдите ЭДС индукции в момент t = 0.45 с.
Ответ: Ответ: ε = 12.5458 В

--- ACCircuitsTask (RLC-цепи) ---
Задача: Решите задачу на цепь переменного тока.

RLC-цепь: R = 967.1 Ом, L = 0.595 Гн, C = 81.7 мкФ, частота f = 850.7 Гц. Найдите импеданс.
Ответ: Ответ: Z = 3321.94 Ом

--- RCCircuitsTask (RC-цепи) ---
Задача: Решите задачу на RC-цепь.

RC-цепь: R = 31.6 кОм, C = 5.42 мкФ. Найдите постоянную времени.
Ответ: Ответ: τ = 0.171272 с = 171.272 мс

--- MagneticForceTask (Сила Лоренца) ---
Задача: Решите задачу на движение заряда в магнитном поле.

Частица с зарядом q = 1.602e-19 Кл движется со скоростью v = 5.796e+06 м/с перпендикулярно полю B = 0.514 Тл. Найдите силу Лоренца.
Ответ: Ответ: F = 4.773e-13 Н


In [15]:
print("=" * 60)
print("ТЕРМОДИНАМИКА")
print("=" * 60)

# Термодинамические циклы
print("\n--- ThermodynamicCyclesTask (Цикл Карно) ---")
task = ThermodynamicCyclesTask(task_type="carnot_efficiency", difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Энтропия
print("\n--- EntropyTask (Энтропия) ---")
task = EntropyTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Фазовые переходы
print("\n--- PhaseTransitionsTask (Фазовые переходы) ---")
task = PhaseTransitionsTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

ТЕРМОДИНАМИКА

--- ThermodynamicCyclesTask (Цикл Карно) ---
Задача: Решите задачу на термодинамический цикл.

Цикл Карно работает между температурами T₁ = 559.0 К (нагреватель) и T₂ = 251.0 К (холодильник). Найдите КПД.
Ответ: Ответ: η = 55.1%

--- EntropyTask (Энтропия) ---
Задача: Решите задачу на энтропию.

При изотермическом расширении 9.0 моль газа при T = 410.0 К объём увеличился в 10 раз. Найдите изменение энтропии.
Ответ: Ответ: ΔS = 172.29 Дж/К

--- PhaseTransitionsTask (Фазовые переходы) ---
Задача: Решите задачу на фазовые переходы.

Сколько теплоты нужно для нагрева 1.52 кг льда от -28.0°C до 38.0°C воды? (c_лёд=2100, c_вода=4186, λ=334 кДж/кг)
Ответ: Ответ: Q = 838.84 кДж


In [16]:
print("=" * 60)
print("ВОЛНЫ И ОПТИКА")
print("=" * 60)

# Эффект Доплера
print("\n--- DopplerEffectTask (Эффект Доплера) ---")
task = DopplerEffectTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Интерференция
print("\n--- InterferenceTask (Интерференция) ---")
task = InterferenceTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Дифракция
print("\n--- DiffractionTask (Дифракция) ---")
task = DiffractionTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Поляризация
print("\n--- PolarizationTask (Поляризация) ---")
task = PolarizationTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

ВОЛНЫ И ОПТИКА

--- DopplerEffectTask (Эффект Доплера) ---
Задача: Решите задачу на эффект Доплера.

Галактика удаляется со скоростью v = 7928.0 км/с. Найдите красное смещение линии с λ₀ = 477.0 нм.
Ответ: Ответ: Δλ = 12.6055 нм, λ = 489.6055 нм

--- InterferenceTask (Интерференция) ---
Задача: Решите задачу на интерференцию света.

Тонкая плёнка толщиной t = 269.0 нм с показателем преломления n = 1.4. Свет с λ = 438.0 нм. Найдите разность хода.
Ответ: Ответ: δ = 753.2 нм

--- DiffractionTask (Дифракция) ---
Задача: Решите задачу на дифракцию света.

Решётка с 9441 штрихами. Найдите разрешающую способность в 2-м порядке.
Ответ: Ответ: R = 18882

--- PolarizationTask (Поляризация) ---
Задача: Решите задачу на поляризацию света.

Неполяризованный свет I₀ = 478.8 проходит через два поляризатора с углом 71.8° между ними. Найдите конечную интенсивность.
Ответ: Ответ: I = 23.35 Вт/м²


In [17]:
print("=" * 60)
print("СОВРЕМЕННАЯ ФИЗИКА")
print("=" * 60)

# Модель Бора
print("\n--- BohrModelTask (Модель атома Бора) ---")
task = BohrModelTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Волны де Бройля
print("\n--- DeBroglieTask (Волны де Бройля) ---")
task = DeBroglieTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Принцип неопределённости
print("\n--- UncertaintyPrincipleTask (Принцип неопределённости) ---")
task = UncertaintyPrincipleTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Радиоактивный распад
print("\n--- RadioactiveDecayTask (Радиоактивный распад) ---")
task = RadioactiveDecayTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

СОВРЕМЕННАЯ ФИЗИКА

--- BohrModelTask (Модель атома Бора) ---
Задача: Решите задачу на модель атома Бора.

Найдите радиус 6-й орбиты электрона в атоме водорода.
Ответ: Ответ: r = 19.044 Å

--- DeBroglieTask (Волны де Бройля) ---
Задача: Решите задачу на волновые свойства частиц.

Найдите длину волны де Бройля для нейтрона с кинетической энергией E = 281.0 эВ.
Ответ: Ответ: λ = 1.7063 пм

--- UncertaintyPrincipleTask (Принцип неопределённости) ---
Задача: Решите задачу на принцип неопределённости Гейзенберга.

Время жизни возбуждённого состояния τ = 4.41e-07 с. Найдите естественную ширину спектральной линии.
Ответ: Ответ: ΔE ≥ 7.4687e-10 эВ

--- RadioactiveDecayTask (Радиоактивный распад) ---
Задача: Решите задачу на радиоактивный распад.

В образце осталось 47.3% исходного изотопа. T₁/₂ = 12.1 лет. Найдите возраст образца.
Ответ: Ответ: t = 13.07 дней


In [18]:
print("=" * 60)
print("ИЗМЕРЕНИЯ И АНАЛИЗ")
print("=" * 60)

# Анализ размерностей
print("\n--- DimensionalAnalysisTask (Анализ размерностей) ---")
task = DimensionalAnalysisTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Распространение погрешностей
print("\n--- ErrorPropagationTask (Погрешности) ---")
task = ErrorPropagationTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

# Перевод единиц
print("\n--- UnitConversionTask (Перевод единиц) ---")
task = UnitConversionTask(difficulty=5, language="ru")
result = task.get_result()
print(f"Задача: {result['problem']}")
print(f"Ответ: {result['final_answer']}")

ИЗМЕРЕНИЯ И АНАЛИЗ

--- DimensionalAnalysisTask (Анализ размерностей) ---
Задача: Проверьте размерность формулы или выведите формулу методом размерностей.

Период колебаний маятника T зависит от длины l и g. Выведите формулу методом размерностей.
Ответ: Ответ: T = C·√(l/g), где C = 2π

--- ErrorPropagationTask (Погрешности) ---
Задача: Рассчитайте погрешность результата измерений.

A = 75.1 ± 4.36, B = 41.2 ± 2.37. Найдите относительную погрешность произведения A·B.
Ответ: Ответ: (3094.12 ± 252.88)

--- UnitConversionTask (Перевод единиц) ---
Задача: Выполните перевод единиц измерения.

Выразите 809.85 т через основные единицы СИ.
Ответ: Ответ: 809.85 т = 809850 кг


## 6. Генерация датасета (пример)

In [21]:
# Генерация 10000 примеров для SFT обучения
generator = DatasetGenerator(output_dir="datasets")

dataset = generator.generate_sft_dataset(
    task_types=None,  # Все типы задач
    num_samples=10000,
    language="ru",
    difficulties=[1,2],  # Средняя сложность
)

# Разделение на train/eval
train, eval = generator.split_dataset(dataset, train_ratio=0.9)

print(f"Train: {len(train)} примеров")
print(f"Eval: {len(eval)} примеров")

Генерация задач: 100%|██████████| 9963/9963 [00:04<00:00, 2154.44задач/s]

Train: 8611 примеров
Eval: 957 примеров


In [22]:
# Сохранение в JSONL формате
generator.save_jsonl(train, "train.jsonl")
generator.save_jsonl(eval, "eval.jsonl")

print("\nФайлы сохранены! Готово для SFT обучения.")

Сохранено 8611 примеров в datasets/train.jsonl
Сохранено 957 примеров в datasets/eval.jsonl

Файлы сохранены! Готово для SFT обучения.


## 7. Chat формат (для Llama/ChatML)

In [ ]:
# Генерация в chat формате
chat_data = generator.generate_chat_dataset(
    task_types=["quantum", "kinematics"],
    num_samples=10,
    language="ru"
)

print("Chat формат:")
print(chat_data[0])

Chat формат:
{'messages': [{'role': 'user', 'content': 'Решите задачу пошагово, объясняя каждый шаг рассуждения.\n\nФотон с длиной волны 5.387e-08 м рассеивается на свободном электроне под углом 180°. Найдите изменение длины волны.'}, {'role': 'assistant', 'content': "Шаг 1: Формула Комптона: Δλ = λc(1 - cos θ), где λc = h/(m_e·c)\nλc = h/(m_e·c) = 2.426e-12 м\nΔλ = λc(1 - cos 180°) = 4.853e-12 м\nλ' = λ + Δλ = 5.388e-08 м\n\nОтвет: Δλ = 4.853e-12 м"}], 'metadata': {'task_type': 'quantum', 'difficulty': 4, 'language': 'ru'}}


## 8. Статистика датасета

In [ ]:
from collections import Counter

# Распределение по типам задач
task_counts = Counter(item['metadata']['task_type'] for item in dataset)

print("Распределение по типам задач:")
for task_type, count in sorted(task_counts.items(), key=lambda x: -x[1])[:10]:
    print(f"  {task_type}: {count}")

# Распределение по сложности
diff_counts = Counter(item['metadata']['difficulty'] for item in dataset)
print("\nРаспределение по сложности:")
for diff, count in sorted(diff_counts.items()):
    print(f"  Сложность {diff}: {count}")

Распределение по типам задач:
  quadratic: 19
  group_theory: 19
  trigonometry: 19
  logarithmic: 19
  calculus: 19
  arithmetic: 19
  waves: 19
  geometry: 19
  statistics: 19
  knights_knaves: 19

Распределение по сложности:
  Сложность 3: 238
  Сложность 5: 230
  Сложность 7: 231
  Сложность 9: 249


## 9. Генерация полного датасета и сохранение в JSONL

При запуске всего ноутбука генерируется полный датасет со всеми 81 типом задач и сохраняется в JSONL формате.

In [ ]:
import os
import json
from datetime import datetime

# Создаём директорию для датасетов
OUTPUT_DIR = "datasets"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Конфигурация генерации
CONFIG = {
    "num_samples_per_type": 50,  # примеров на тип задачи
    "languages": ["ru"],         # языки
    "difficulties": [3, 5, 7, 9], # уровни сложности
    "train_ratio": 0.9,          # доля train
}

print("=" * 60)
print("ГЕНЕРАЦИЯ ПОЛНОГО ДАТАСЕТА")
print("=" * 60)
print(f"\nКонфигурация:")
print(f"  Примеров на тип задачи: {CONFIG['num_samples_per_type']}")
print(f"  Языки: {CONFIG['languages']}")
print(f"  Уровни сложности: {CONFIG['difficulties']}")
print(f"  Train/Eval split: {CONFIG['train_ratio']*100:.0f}%/{(1-CONFIG['train_ratio'])*100:.0f}%")

In [ ]:
# Генератор датасетов
generator = DatasetGenerator(output_dir=OUTPUT_DIR)

# ВСЕ 81 тип задач (ALL_TASK_GENERATORS теперь включает и физические)
all_task_types = list(ALL_TASK_GENERATORS.keys())

print(f"\nВсего типов задач: {len(all_task_types)}")
print(f"Ожидаемое количество примеров: ~{len(all_task_types) * CONFIG['num_samples_per_type']}")

# Примеры категорий задач:
print("\nПримеры категорий:")
print(f"  Reasoning: sudoku, zebra_puzzle, river_crossing, tower_of_hanoi, water_jug, nim_game")
print(f"  Физика: projectile_motion, rotational_dynamics, bohr_model, doppler_effect, ...")
print(f"  Математика: quadratic, calculus, statistics, combinatorics, ...")

In [ ]:
import random

# Генерация датасета
print("\nГенерация датасета...")
print(f"Все типы задач: {len(ALL_TASK_GENERATORS)}")

full_dataset = []

for lang in CONFIG["languages"]:
    print(f"\n  Язык: {lang}")
    
    # Генерация через DatasetGenerator (ВСЕ 81 тип задач)
    all_tasks = list(ALL_TASK_GENERATORS.keys())
    
    sft_data = generator.generate_sft_dataset(
        task_types=all_tasks,
        num_samples=len(all_tasks) * CONFIG["num_samples_per_type"],
        language=lang,
        difficulties=CONFIG["difficulties"],
    )
    full_dataset.extend(sft_data)
    print(f"    Сгенерировано: {len(sft_data)} примеров")

print(f"\n  Всего сгенерировано: {len(full_dataset)} примеров")

In [ ]:
# Перемешиваем датасет
random.shuffle(full_dataset)

# Разделяем на train/eval
split_idx = int(len(full_dataset) * CONFIG["train_ratio"])
train_data = full_dataset[:split_idx]
eval_data = full_dataset[split_idx:]

print(f"\nРазделение датасета:")
print(f"  Train: {len(train_data)} примеров")
print(f"  Eval: {len(eval_data)} примеров")

In [ ]:
# Сохранение в JSONL
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

train_file = os.path.join(OUTPUT_DIR, f"reasoning_train_{timestamp}.jsonl")
eval_file = os.path.join(OUTPUT_DIR, f"reasoning_eval_{timestamp}.jsonl")

def save_jsonl(data, filepath):
    """Сохраняет данные в JSONL формате."""
    with open(filepath, 'w', encoding='utf-8') as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    return filepath

save_jsonl(train_data, train_file)
save_jsonl(eval_data, eval_file)

print(f"\n" + "=" * 60)
print("ДАТАСЕТ СОХРАНЁН!")
print("=" * 60)
print(f"\nФайлы:")
print(f"  Train: {train_file}")
print(f"  Eval: {eval_file}")
print(f"\nСтатистика:")
print(f"  Всего примеров: {len(full_dataset)}")
print(f"  Train: {len(train_data)}")
print(f"  Eval: {len(eval_data)}")

In [ ]:
# Финальная статистика
from collections import Counter

task_counts = Counter(item['metadata']['task_type'] for item in full_dataset)
diff_counts = Counter(item['metadata']['difficulty'] for item in full_dataset)

print("\nРаспределение по типам задач (топ-15):")
for task_type, count in sorted(task_counts.items(), key=lambda x: -x[1])[:15]:
    print(f"  {task_type}: {count}")

print(f"\n  ... и ещё {len(task_counts) - 15} типов задач")

print("\nРаспределение по сложности:")
for diff, count in sorted(diff_counts.items()):
    print(f"  Сложность {diff}: {count}")

print("\n" + "=" * 60)
print("ГОТОВО! Датасет готов для обучения LLM.")
print("=" * 60)